## Import

In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from setproctitle import setproctitle
from sklearn.model_selection import train_test_split
from itertools import combinations
from rank_bm25 import BM25L

import torch
import torch.nn as nn
import random
import time
import datetime
import os, re
import argparse

## Load Train / Test dataset

In [ ]:
# 추가로 주어지는 Python Code 데이터들을 활용하여 새로운 Pair 쌍을 생성하여 더 많은 데이터로 학습할 수 있습니다.
# 베이스라인에서는 이미 Pair 쌍으로 구축되어 주어지는 sample 데이터들로 검증을 진행합니다.
val = pd.read_csv("./sample_train.csv")
val.head()

In [ ]:
test = pd.read_csv("./test.csv")
test.head()

## Define Model (CountVectorizer+CosineSimilarity)

In [ ]:
class BaselineModel():
    def __init__(self, threshold=0.5):
        super(BaselineModel, self).__init__()
        self.threshold = threshold # 유사도 임계값
        self.vocabulary = set()

    def get_vectorizer(self):
        return CountVectorizer(vocabulary=list(self.vocabulary))
        
    def fit(self, code):
        # 입력 받은 학습 코드로 부터 vectorizer를 fit 시킵니다.
        temp_vectorizer = CountVectorizer()
        temp_vectorizer.fit(code)
        # fit 호출 마다 vectorizer에 활용할 vocabulary 업데이트
        self.vocabulary.update(temp_vectorizer.get_feature_names_out())
        # fit 호출 마다 vectorizer 업데이트
        self.vectorizer = self.get_vectorizer()
    
    def predict_proba(self, code1, code2):
        # 입력 받은 코드 쌍으로 부터 vectorizer를 통해 vector화 합니다.
        code1_vecs = self.vectorizer.transform(code1)
        code2_vecs = self.vectorizer.transform(code2)
        
        preds = []
        # 각각의 코드 쌍(=벡터 쌍)으로부터 cosine-similarity를 구합니다.
        for code1_vec, code2_vec in tqdm(zip(code1_vecs, code2_vecs)):
            preds.append(cosine_similarity(code1_vec, code2_vec))
        
        preds = np.reshape(preds, len(preds))
        print('Done.')
        # 각 코드 쌍들의 유사도를 반환
        return preds
    
    def predict(self, code1, code2):
        preds = self.predict_proba(code1, code2)
        # cosine-similarity (유사도)가 설정한 임계값(Threshold=0.5)보다 높다면 유사하다 : 1, 아니라면 유사하지 않다 : 0
        preds = np.where(preds>self.threshold, 1, 0)
        # 각 코드 쌍들의 유사도를 Threshold를 통해 유사함을 판별 (이진분류)
        return preds

## Model(Vectorizer) Fit

In [ ]:
train_code_paths = glob.glob('./train_code/*/*.cpp')

In [ ]:
def read_cpp_code(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [ ]:
# 모델 선언
model = BaselineModel(threshold=0.5)

In [ ]:
for path in tqdm(train_code_paths):
    code = read_cpp_code(path)
    model.fit([code])

In [ ]:
len(model.vocabulary)

## Validation

In [ ]:
def get_accuracy(gt, preds):
    return (gt == preds).mean()

In [ ]:
val_preds = model.predict(val['code1'], val['code2'])

In [ ]:
print(get_accuracy(val['similar'].values, val_preds))

## Inference

In [ ]:
# 모델 추론
preds = model.predict(test['code1'], test['code2'])

## Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['similar'] = preds
submission.to_csv('./base_submit.csv', index=False)